# Basic Maths

In [ ]:
# Libraries to import

import torch

import sympy as sp

# Create input tensor with gradient tracking

x = torch.tensor([2.0, 3.0], requires_grad=True)



# Define a differentiable function: f(x1, x2) = x1^2 + 3*x1*x2 + x2^2

y = x[0]**2 + 3*x[0]*x[1] + x[1]**2



# Compute gradients

y.backward()



# Gradients each input

grad_x1 = x.grad[0]  # ∂f/∂x1

grad_x2 = x.grad[1]  # ∂f/∂x2



# Print results

print("PyTorch gradients:")

print("Gradient ∂f/∂x1:", grad_x1)

print("Gradient ∂f/∂x2:", grad_x2)



# Define symbolic variables

x1, x2 = sp.symbols('x1 x2')



# Define the same function symbolically

f = x1**2 + 3*x1*x2 + x2**2



# Compute symbolic derivatives

df_dx1 = sp.diff(f, x1)

df_dx2 = sp.diff(f, x2)



# Show the derivative formulas

print("SymPy derivative formulas:")

print("∂f/∂x1 =", df_dx1)

print("∂f/∂x2 =", df_dx2)



# Evaluate derivatives at a specific point (x1=2, x2=3)

grad_x1_sym = df_dx1.evalf(subs={x1:2, x2:3})

grad_x2_sym = df_dx2.evalf(subs={x1:2, x2:3})



# Print numerical results

print("SymPy symbolic gradients evaluated at (x1=2, x2=3):")

print("Gradient x1:", grad_x1_sym)

print("Gradient x2:", grad_x2_sym)

SyntaxError: invalid non-printable character U+200B (2055092030.py, line 3)

In [11]:
# 3pps
import torch

# Example 1: Quadratic Function
# y = x², dy/dx = 2x
x = torch.tensor(3.0, requires_grad=True)
y = x**2
y.backward()
print(f"y = x² | x={x.item()}, dy/dx={x.grad.item()}")

# Example 2: Multiple Variables
# z = 2a + 3b, dz/da = 2, dz/db = 3
a = torch.tensor(4.0, requires_grad=True)
b = torch.tensor(5.0, requires_grad=True)
z = 2*a + 3*b
z.backward()
print(f"z = 2a + 3b | dz/da={a.grad.item()}, dz/db={b.grad.item()}")

# Example 3: Chain Rule
# y = (2x + 1)², dy/dx = 4(2x + 1)
x = torch.tensor(3.0, requires_grad=True)
y = (2*x + 1)**2
y.backward()
print(f"y = (2x+1)² | x={x.item()}, dy/dx={x.grad.item()}")

y = x² | x=3.0, dy/dx=6.0
z = 2a + 3b | dz/da=2.0, dz/db=3.0
y = (2x+1)² | x=3.0, dy/dx=28.0


In [ ]:
# Example 4: Linear Regression
# y = w·x + b, dy/dx = w
x = torch.tensor([2.0, 3.0], requires_grad=True)
w = torch.tensor([0.5, -1.0])
b = 2.0
y = w[0]*x[0] + w[1]*x[1] + b
y.backward()
print(f"Linear | dy/dx1={x.grad[0].item()}, dy/dx2={x.grad[1].item()}")

# Example 5: Logistic Regression
# y = σ(w·x + b), dy/dx = σ'(z)·w
x = torch.tensor([2.0, 3.0], requires_grad=True)
z = w[0]*x[0] + w[1]*x[1] + b
y = torch.sigmoid(z)
y.backward()
print(f"Logistic | dy/dx1={x.grad[0].item():.4f}, dy/dx2={x.grad[1].item():.4f}")

Linear | dy/dx1=0.5, dy/dx2=-1.0
Logistic | dy/dx1=0.1250, dy/dx2=-0.2500
tensor(0.5000, grad_fn=<SigmoidBackward0>)


In [16]:
# 3pps
# Libraries to import
import torch
import torch.nn.functional as F

# Input features
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)

# Weights and bias for 3 classes
W = torch.tensor([[0.2, -0.5, 0.3],   # weights for x[0]
                  [0.4, 0.1, -0.2],   # weights for x[1]
                  [0.1, 0.3, 0.2]], requires_grad=False)
b = torch.tensor([0.0, 0.0, 0.0])

# Linear scores for each class: s = W^T x + b
logits = torch.matmul(x, W) + b  # shape [3]

# Apply Softmax to get probabilities
probs = F.softmax(logits, dim=0)

# Pick the predicted class probability
pred_class_idx = probs.argmax()
top_prob = probs[pred_class_idx]

# Compute gradients w.r.t input
top_prob.backward()

# Print results
print("Multiclass Classification | Probabilities:", probs.detach().numpy())
print("Predicted class index:", pred_class_idx.item())
print("Gradients inputs:", x.grad.detach().numpy())

Multiclass Classification | Probabilities: [0.51389724 0.25519383 0.23090893]
Predicted class index: 0
Gradients inputs: [ 0.07993404  0.1105411  -0.03809503]
